Organization:
- Work
  - 1 test: defining functions for part 1, testing on test input
  - 1 run: getting answer for part 1
  - 2 test: ...
  - 2 run: ...
- Utilities: functions I think might help parse general inputs
- Inputs: where I define the test (_t_) and problem (_s_) inputs

# Work

## 1 test

In [ ]:
import numpy as np

In [5]:
paths = apply_func(split(t, block_char = '\n', line_char = ' -> '), eval, True)
paths

[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]]

In [80]:
class Cave():
    def __init__(self, paths):
        # Get bounds of the cave
        X = [x for path in paths for x, y in path]
        Y = [y for path in paths for x, y in path]
        y1 = 0
        y2 = max(Y)
        x1 = min(X)
        x2 = max(X)

        # For funsies
        assert x1 <= 500
        assert 500 <= x2
        assert 0 <= y2

        # Create an array for the cave state, with the indices of the input location
        self.cave = np.zeros((y2+1, x2-x1+1), dtype=str)
        self.n, self.m = self.cave.shape
        self.xin, self.yin = 500 - x1, 0
        
        # Set symbols
        for i in range(self.n):
            for j in range(self.m):
                self.cave[i,j] = '.'
        self.cave[self.yin, self.xin] = '+'
        
        # Add rock
        for path in paths:
            # Fill in the start of the path
            j, i = path[0]
            self.cave[i, j-x1] = '#'
            
            # Fill in the segments
            for k in range(1, len(path)):
                # Start/end of a segment
                j, i = path[k-1]
                jj, ii = path[k]
                
                # Order for slicing
                if ii < i:
                    i, ii = ii, i
                if jj < j:
                    j, jj = jj, j
                    
                # Fill it in
                self.cave[i:ii+1, (j-x1):(jj-x1+1)] = '#'
    
    # If it worked, return True
    # If the sand falls off the map, return False
    def add_sand(self):
        # Location of the sand
        i, j = self.yin, self.xin
        
        # Keep updating it until we stop moving or fall off
        while True:
            # Down
            if i + 1 == self.n:
                # Fall off
                r = False
                break
            if self.cave[i+1,j] == '.':
                # Drop
                i += 1
                continue
            
            # Diag left
            if j == 0:
                # Fall off
                r = False
                break
            if self.cave[i+1,j-1] == '.':
                # Drop
                i += 1
                j -= 1
                continue
            
            # Diag right
            if j + 1 == self.m:
                # Fall off
                r = False
                break
            if self.cave[i+1,j+1] == '.':
                # Drop
                i += 1
                j += 1
                continue
            
            # If we reached here, we stay in place
            self.cave[i,j] = 'o'
            r = True
            break
            
        return r
    
    def print(self):
        # Just print each row
        for i in range(self.n):
            row = list(self.cave[i, :])
            print(''.join(row))

In [84]:
C = Cave(paths)
C.print()

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.


In [85]:
C.add_sand()
C.print()

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
......o.#.
#########.


In [86]:
C.add_sand()
C.print()

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
.....oo.#.
#########.


In [87]:
[C.add_sand() for i in range(3)]
C.print()

......+...
..........
..........
..........
....#...##
....#...#.
..###...#.
......o.#.
....oooo#.
#########.


In [88]:
[C.add_sand() for i in range(17)]
C.print()

......+...
..........
......o...
.....ooo..
....#ooo##
....#ooo#.
..###ooo#.
....oooo#.
...ooooo#.
#########.


In [89]:
C = Cave(paths)
grains = 0
while C.add_sand():
    grains += 1
grains

24

## 1 run

In [90]:
paths = apply_func(split(s, block_char = '\n', line_char = ' -> '), eval, True)

In [91]:
C = Cave(paths)
C.print()

............................+..........................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.....................#..........................................

In [92]:
grains = 0
while C.add_sand():
    grains += 1
grains

755

In [93]:
C.print()

............................+..........................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.......................................................................
.....................#.....oo...................................

## 2 test

In [102]:
class BigCave():
    def __init__(self, paths):
        # Get bounds of the cave
        X = [x for path in paths for x, y in path]
        Y = [y for path in paths for x, y in path]
        y1 = 0
        y2 = max(Y)
        x1 = min(X)
        x2 = max(X)
        
        # Extend horizontally to be at least y2 + 10 in each direction from the sand inflow
        # This is more than enough to be "infinite" for our purposes
        x1 = min(x1, 500 - y2 - 10)
        x2 = max(x2, 500 + y2 + 10)

        # For funsies
        assert x1 <= 500
        assert 500 <= x2
        assert 0 <= y2

        # Create an array for the cave state, with the indices of the input location
        # A bit taller, to include the "infinite" floor
        self.cave = np.zeros((y2+3, x2-x1+1), dtype=str)
        self.n, self.m = self.cave.shape
        self.xin, self.yin = 500 - x1, 0
        
        # Set symbols
        for i in range(self.n):
            for j in range(self.m):
                self.cave[i,j] = '.'
        self.cave[self.yin, self.xin] = '+'
        
        # Set floor
        i = self.n - 1
        for j in range(self.m):
            self.cave[i,j] = '#'
        
        # Add rock
        for path in paths:
            # Fill in the start of the path
            j, i = path[0]
            self.cave[i, j-x1] = '#'
            
            # Fill in the segments
            for k in range(1, len(path)):
                # Start/end of a segment
                j, i = path[k-1]
                jj, ii = path[k]
                
                # Order for slicing
                if ii < i:
                    i, ii = ii, i
                if jj < j:
                    j, jj = jj, j
                    
                # Fill it in
                self.cave[i:ii+1, (j-x1):(jj-x1+1)] = '#'
    
    # If it worked, return True
    # If the sand falls off the map, return False
    def add_sand(self):
        # Location of the sand
        i, j = self.yin, self.xin
        
        # Keep updating it until we stop moving or fall off
        while True:
            # Down
            if i + 1 == self.n:
                # Fall off
                r = False
                break
            if self.cave[i+1,j] == '.':
                # Drop
                i += 1
                continue
            
            # Diag left
            if j == 0:
                # Fall off
                r = False
                break
            if self.cave[i+1,j-1] == '.':
                # Drop
                i += 1
                j -= 1
                continue
            
            # Diag right
            if j + 1 == self.m:
                # Fall off
                r = False
                break
            if self.cave[i+1,j+1] == '.':
                # Drop
                i += 1
                j += 1
                continue
            
            # If we reached here, we stay in place
            self.cave[i,j] = 'o'
            r = True
            break
            
        return r
    
    def print(self):
        # Just print each row
        for i in range(self.n):
            row = list(self.cave[i, :])
            print(''.join(row))

In [103]:
paths = apply_func(split(t, block_char = '\n', line_char = ' -> '), eval, True)
paths

[[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]]

In [104]:
C = BigCave(paths)
C.print()

...................+...................
.......................................
.......................................
.......................................
.................#...##................
.................#...#.................
...............###...#.................
.....................#.................
.....................#.................
.............#########.................
.......................................
#######################################


In [105]:
C.add_sand()
C.print()

...................+...................
.......................................
.......................................
.......................................
.................#...##................
.................#...#.................
...............###...#.................
.....................#.................
...................o.#.................
.............#########.................
.......................................
#######################################


In [107]:
C = BigCave(paths)
[C.add_sand() for i in range(92)]
C.print()
C.add_sand()
C.print()

...................+...................
..................ooo..................
.................ooooo.................
................ooooooo................
...............oo#ooo##o...............
..............ooo#ooo#ooo..............
.............oo###ooo#oooo.............
............oooo.oooo#ooooo............
...........oooooooooo#oooooo...........
..........ooo#########ooooooo..........
.........ooooo.......ooooooooo.........
#######################################
...................o...................
..................ooo..................
.................ooooo.................
................ooooooo................
...............oo#ooo##o...............
..............ooo#ooo#ooo..............
.............oo###ooo#oooo.............
............oooo.oooo#ooooo............
...........oooooooooo#oooooo...........
..........ooo#########ooooooo..........
.........ooooo.......ooooooooo.........
#######################################


In [108]:
paths = apply_func(split(t, block_char = '\n', line_char = ' -> '), eval, True)
C = BigCave(paths)
grains = 0
while C.cave[C.yin, C.xin] == '+':
    C.add_sand()
    grains += 1
grains

93

## 2 run

In [109]:
paths = apply_func(split(s, block_char = '\n', line_char = ' -> '), eval, True)
C = BigCave(paths)
grains = 0
while C.cave[C.yin, C.xin] == '+':
    C.add_sand()
    grains += 1
grains

29805

In [110]:
C.print()

..........................................................................................................................................................................................o..........................................................................................................................................................................................
.........................................................................................................................................................................................ooo.........................................................................................................................................................................................
........................................................................................................................................................................................ooooo...............................................................

Copy/paste that into Notepad and zoom out, it's pretty

# Utilities

In [1]:
# Remove initial/final \n characters
def clean(s):
    return s[1:-1]

# Split at \n characters
# If there are \n\n characters, split into blocks too
def split(s, block_char = '\n\n', line_char = '\n'):
    out = [block.split(line_char) for block in clean(s).split(block_char)]
    if len(out) == 1:
        return out[0]
    else:
        return out

# Apply a function(s) to a list or "block" data (2-level list)
def apply_func(data, func, nested=False):
    if not isinstance(func, list):
        func = [func]
        
    def _func(x):
        for f in func:
            x = f(x)
        return x
        
    if nested:
        return [[_func(x) for x in block] for block in data]
    else:
        return [_func(x) for x in data]

# Split, parsing everything as ints
def split_int(s):
    return apply_func(split(s), int)

# Split, parsing everything as float
def split_float(s):
    return apply_func(split(s), float)

# Inputs

In [2]:
t = """
498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9
"""

In [3]:
s = """
529,71 -> 529,72 -> 539,72 -> 539,71
...
527,96 -> 527,98 -> 525,98 -> 525,106 -> 536,106 -> 536,98 -> 530,98 -> 530,96
"""